In [20]:
import os
import requests
import ollama
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [7]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [8]:
technical_prompt = "Du bist ein Experte in der Softwareentwicklung für die Sprache Python, C# und C++. Du unterstützt die Softwareentwickler dabei ihren Code zu verbessern und bewertest ihren Programierstil."

In [28]:
user_prompt = "Analysiere und bewerte folgenden C# code und erstelle einen Bericht in Markdown mit Verbesserungsvorschläge (ohne Code Beispiele) und einer Bewertung nach Entwickler Level (Beginer, Normal, Senior, Expert):/n"
user_prompt += """using System.IdentityModel.Tokens.Jwt;
                using System.Security.Claims;
                using API.DTOs;
                using Blazored.LocalStorage;
                using Microsoft.AspNetCore.Components.Authorization;
                
                namespace Client.Components.Provider;
                
                public class CustomAuthenticationStateProvider : AuthenticationStateProvider
                {
                    private readonly ILocalStorageService _localStorage;
                    private readonly HttpClient _httpClient;
                
                    public CustomAuthenticationStateProvider(
                        ILocalStorageService localStorage,
                        HttpClient httpClient)
                    {
                        _localStorage = localStorage;
                        _httpClient = httpClient;
                    }
                
                    public override async Task<AuthenticationState> GetAuthenticationStateAsync()
                    {
                        var user = await _localStorage.GetItemAsync<UserDto>("user");
                
                        if (user == null || string.IsNullOrEmpty(user.Token))
                        {
                            return new AuthenticationState(new ClaimsPrincipal(new ClaimsIdentity()));
                        }
                
                        SetAuthorizationHeader(user.Token);
                
                        var claims = ParseClaimsFromJwt(user.Token);
                        var identity = new ClaimsIdentity(claims, "jwt");
                        var principal = new ClaimsPrincipal(identity);
                
                        return new AuthenticationState(principal);
                    }
                
                    public async Task SetAuthenticationState(UserDto user)
                    {
                        if (user != null)
                        {
                            SetAuthorizationHeader(user.Token);
                            var claims = ParseClaimsFromJwt(user.Token);
                            var identity = new ClaimsIdentity(claims, "jwt");
                            var principal = new ClaimsPrincipal(identity);
                            NotifyAuthenticationStateChanged(Task.FromResult(new AuthenticationState(principal)));
                        }
                        else
                        {
                            _httpClient.DefaultRequestHeaders.Authorization = null;
                            NotifyAuthenticationStateChanged(Task.FromResult(new AuthenticationState(new ClaimsPrincipal())));
                        }
                    }
                
                    private void SetAuthorizationHeader(string token)
                    {
                        _httpClient.DefaultRequestHeaders.Authorization = 
                            new System.Net.Http.Headers.AuthenticationHeaderValue("Bearer", token);
                    }
                
                    private IEnumerable<Claim> ParseClaimsFromJwt(string jwt)
                    {
                        var handler = new JwtSecurityTokenHandler();
                        var token = handler.ReadJwtToken(jwt);
                        return token.Claims;
                    }
                }"""

In [40]:
messages=[
            {"role": "system", "content": technical_prompt},
            {"role": "user", "content": user_prompt}
          ]

In [45]:
def stream_open_ai_answer():
    load_dotenv(override=True)
    api_key = os.getenv('OPENAI_API_KEY')
    
    if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
        print("API key looks good so far")
    else:
        print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
        
    openai = OpenAI()
    stream_open_ai(openai,MODEL_GPT)


In [46]:
def stream_open_ai_ollama_answer():
    load_dotenv(override=True)
    ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
    
    stream_open_ai(ollama_via_openai, MODEL_LLAMA)

In [47]:
def stream_open_ai(openai, model):
    stream = openai.chat.completions.create(
        model=model,
        messages=messages,
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [48]:
stream_open_ai_ollama_answer()

**Bericht: Verbesserungs- und Bewertungsempfehlungen für C#-Code**

**Übersicht**

Der angeführte Code teilt sich in verschiedene Abfragen, die das Verhalten der Authentifizierung in einem ASP.NET Core-Anwendungen regieren. Der Schwerpunkt liegt auf der Implementierung eines Custom-Authentication-State-Providers. 

### Verbesserungsvorschläge

*   **Vermeidung von Konfigurationsschwachstellen**: Die Verwendung von Benutzerflussvariablen (UserDto) in `SetAuthenticationState` und das anschließende Löschen der Authentifizierung, wenn kein Nutzer vorliegt, führt zu Sicherheitsrisiken. Statt des `ILocalStorageService` könnte eine einfache Methode von Windows-Datenbank oder einen authentifizierten Token speichern.
*   **Erweiterung der Error-Handschrage**: Der Code verwendet keine detaillere Meldung bei Fehlern und Ausnahmen, die während der Verarbeitung der JWT-Bestandteile entstehen. 
*   **Verbesserung der Einheitlichkeit der Logik**: Die Methode `ParseClaimsFromJwt` könnte zu einem separaten Klasse übernommen werden. 

### Bewertung nach Entwickler Level

**Fähigkeitsebene:** Senior/Expert

**Warum?**
Der angeführte Code zeigt eine klare und korrekte Implementierung eines Authentication-Providers, der komplexen Logik von Windows-Datenspeichern oder authentifizierten Tokens nutzt. 

Die Implementierung ist auf prägnante Art die Klärung von Authentifizierungsproblemen im ASP.NET Core Framework ausgeführt. Die Implementierung des JWT-Handhabers und der Token-Verarbeitung ist ein wichtiger Bestandteil dieser Analyse.

Der Schwerpunkt auf der Verwendung lokaler Speicherntechnologie für authentifizierte Datei-Speicherverwaltung ermöglicht mehr sichere, prägnante und umfassende Code-Entwicklungsphasen.

Die Implementierung bietet klare und komplexe Logik - ein wichtiger indikator für ein Senior/Experten-Ebene für Software-Entwicklung.

In [49]:
stream_open_ai_answer()

API key looks good so far


# Code-Analyse und Verbesserungsvorschläge für den C# Code

## Zusammenfassung
Der bereitgestellte C# Code implementiert einen benutzerdefinierten Authentifizierungszustandsanbieter für eine Blazor-Anwendung. Er verwendet lokale Speicherung für die Speicherung des Benutzers und verwaltet die Autorisierung durch JWT (JSON Web Tokens). 

## Bewertung
**Entwicklerlevel: Normal**

### Bewertungsgründe:
- **Struktur:** Der Code hat eine klare Struktur, die die Lesbarkeit fördert. Klassen und Methoden sind klar benannt und haben eine deutliche Verantwortung.
- **Asynchrone Programmierung:** Die Verwendung von `async` und `await` ist angemessen und zeigt ein grundlegendes Verständnis von asynchroner Programmierung in C#.
- **Verwendung von Libraries:** Der Code nutzt Standardbibliotheken und gängige Praktiken zur JWT-Token-Verarbeitung.

## Verbesserungsvorschläge

1. **Fehlerbehandlung:**
   - Es fehlt eine umfassende Fehlerbehandlung für Situationen, in denen das Token ungültig ist oder beim Zugriff auf lokale Speicherung Fehler auftreten. Implementierung von Try-Catch-Blöcken kann helfen, unerwartete Fehler zu vermeiden.

2. **Token Validierung:**
   - Vor der Nutzung des Tokens sollte eine Validierung des JWT durchgeführt werden, um sicherzustellen, dass es nicht abgelaufen oder ungültig ist. Dies sollte in die `ParseClaimsFromJwt` Methode integriert werden.

3. **Verwendung von Konstanten:**
   - Anstelle von „Bearer“ in der `SetAuthorizationHeader` Methode als Hardcodierung könnten Konstanten verwendet werden, um Änderungen in der Zukunft zu vereinfachen und den Code lesbarer zu machen.

4. **Sicherheit:**
   - Die Implementierung könnte durch zusätzliche Sicherheitsmaßnahmen verbessert werden, wie zum Beispiel die Absicherung gegen Cross-Site Script (XSS) und CSRF (Cross-Site Request Forgery). Hierzu wäre eine Analyse der gesamten Anwendung notwendig.

5. **Methoden Kohäsion:**
   - Die Methode `SetAuthenticationState` hat mehrere Verantwortlichkeiten. Es könnte sinnvoll sein, die Logik für das Erstellen des `ClaimsPrincipal` in eine separate Methode auszulagern, um die Lesbarkeit und Wartbarkeit des Codes zu verbessern.

6. **Async/Await Konsistenz:**
   - Der erste Teil der `SetAuthenticationState` Methode könnte durch eine asynchrone Implementierung dieser Logik verbessert werden, um Konsistenz im Code zu gewährleisten.

7. **Dokumentation:**
   - Es gibt wenig bis keine Kommentierung oder XML-Dokumentation. Das Hinzufügen von Kommentaren könnte das Verständnis des Codes für andere Entwickler erleichtern.

8. **Unit Tests:**
   - Es fehlen Tests zur Überprüfung der Authentifizierungslogik. Der Code sollte so gestaltet werden, dass er leicht testbar ist, und Unit Tests sollten hinzugefügt werden, um sicherzustellen, dass die Logik korrekt funktioniert.

## Fazit
Insgesamt zeigt der Code ein solides Grundverständnis von C# und der ASP.NET-Blazor-Architektur. Dennoch gibt es Raum für Verbesserungen bei der Fehlerbehandlung, Sicherheit und Lesbarkeit. Die oben genannten Verbesserungsvorschläge können dazu beitragen, die Qualität und Wartbarkeit des Codes zu erhöhen.